___
# Projeto 2 - Ciência dos Dados 2020.2
Grupo: Lucas Kang, Lucas Franco Florentino
___

# Descrição do projeto



O aumento no número de suícios de 200% a 400% nos últimos 20 anos vem se tornando um problema de saúde pública em vários países ao redor do mundo. Dessa forma, várias pesquisas foram desenvolvidas com a intenção de entender esse problema e suas relações externas. Quando relacionado a idade e o sexo com a frequência dos casos de suicídio, existe uma variação interessante de país para país, além disso, no caso dos homens, ocorrem geralmente entre 25 e 35 anos, já nas mulheres entre 18 e 30 anos. Existem alguns indicadores de suicídio, porém, a maioria deles referem ao desemprego, situação financeira do país e consequentemente a presença de doenças psiquiátricas e crônicas. 

   Dessa forma, o projeto tem como função principal observar o relacionamento entre vários indicadores como idade, sexo, situação econômica do país e a geração, com o número de suicídios de um país. Dessa forma, será examinado os resultados e as conexões com o número de suicídios. 

## Objetivo do projeto:

- A partir de diversos fatores como o ano, sexo, faixa etária e situação econômica de um país, é possível prever a quantidade de pessoas que irão cometer suicídio neste ano?  
- O target é quantitativo, já que o objetivo é prever uma informação numérica.

### Importando as bibliotecas:

In [12]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
from math import *
from scipy import *
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from scipy import stats

### Importando a base de dados:

In [2]:
suicidios = pd.read_csv('suicide_rates.csv')

# apagar depois

## coisas a serem estudadas

- multiple linear regression
- random forest regression
- variável qualitativa precisa ser transformada em dummies
- pd.get_dummies() -- estudar e ler a documentação do comando

# Explicando o dataset:

A base de dados suicide_rates.csv contém um conjunto de informações com cerca de 102 países ao passar de 33 anos. Dessa forma, o código irá analisar uma base de treinamento, e depois, aplicar um teste nos classificadores, onde um deles tem como princípio a regressão múltipla e o outro de #aaaaaa, para então provar sua eficiência em prever a quantidade de suicídios em um ano. 
A base de dados contém os seguintes dados:

- Country: País
- Year: Ano
- Sex: Sexo
- Age: Idade
- Suicides_no: Número de suicídios no país, no ano, no sexo e na faixa etária presente na mesma linha
- Population: População do país, do ano, do sexo e da faixa etária presente na mesma linha
- Suicides/100kpop: Taxa de suicideos a cada 100.000 pessoas com os filtros presentes na mesma linha
- Country-year: País e ano
- HDI for year: Índice de desenvolvimento humano por ano
- Gdp_for_year (): Produto interno bruto por ano em dolar
- Gdp_per_capita (): Produto interno bruto per capita em dolar
- Generation: Geração do indivíduo


Nossa base de dados:
- 

In [3]:
suicidios.head(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


---
# Organizando os dados

Os dados foram organizados em novas colunas e alguns valores foram traduzidos para números que possa ser utilizados pelo modelo para realizar a predição.
Algumas destas novas colunas foram:
- criação de dummies para armazenar a faixa etária de cada linha;
- criação da coluna de classificação econômica;
- traduzir a coluna de geração de valores categóricos para valores numéricos;
- traduzir a coluna de sexo de valores categóricos para valores numéricos.

In [4]:
faixa_etaria = pd.get_dummies(suicidios['age'])
suicidios = pd.concat([suicidios, faixa_etaria], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,0,0,1,0,0,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,0,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,0,1,0,0,0,0


In [5]:
# comparar gdp_per_capita pra determinar se o país naquele ano era considerado Rico, Em desenvolvimento, ou Pobre (e.g: resultado maior que 30000 dol = rico, menor que 5000 dol/pessoa = probre)
# Utilizando a base de dados do IMF, foi feito uma divisão do gdp do país pelo gdp de Luxemburgo do ano de 2020 (USD 109602). 
# Os países que se encontraram com porcentagem abaixo de 30% foram consideradas pobres, de 31% a 60%, em desenvolvimento, e acima de 61%, rico.
# A divisão categórica foi feita a fim de analizar o efeito da situação econômica na mortalidade de pessoas por suicídio.
'''
Legenda:
    Pobre: 3
    Em Desenvolvimento: 2
    Rico: 1
'''
suicidios['classificação econômica'] = np.where((suicidios['gdp_per_capita ($)']/109602)>0.61, 1, 
    (np.where((suicidios['gdp_per_capita ($)']/109602)<0.3, 3,2)))

In [6]:
# criação de uma dummies para armazenar a geração em formato numérico
gen = pd.get_dummies(suicidios['generation'])
suicidios = pd.concat([suicidios, gen], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,5-14 years,55-74 years,75+ years,classificação econômica,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,1,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,0,0,0,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,1,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,0,0,1,3,0,1,0,0,0,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,1,0,0,0,0,0


In [7]:
# criação de uma dummies para armazenar o sexo em formato numérico
suicidios.loc[suicidios['sex']=='male','sexo'] = 1
suicidios.loc[suicidios['sex']=='female','sexo'] = 0
#suicidios = pd.concat([suicidios, sexo], axis = 1)
suicidios['sexo'] = suicidios['sexo'].astype('int64')
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,55-74 years,75+ years,classificação econômica,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent,sexo
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,0,0,3,0,0,1,0,0,0,1
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,0,3,0,0,0,0,0,1,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,0,0,3,0,0,1,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,0,1,3,0,1,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,0,3,1,0,0,0,0,0,1


In [8]:
suicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  27820 non-null  object 
 1   year                     27820 non-null  int64  
 2   sex                      27820 non-null  object 
 3   age                      27820 non-null  object 
 4   suicides_no              27820 non-null  int64  
 5   population               27820 non-null  int64  
 6   suicides/100k pop        27820 non-null  float64
 7   country-year             27820 non-null  object 
 8   HDI for year             8364 non-null   float64
 9    gdp_for_year ($)        27820 non-null  object 
 10  gdp_per_capita ($)       27820 non-null  int64  
 11  generation               27820 non-null  object 
 12  15-24 years              27820 non-null  uint8  
 13  25-34 years              27820 non-null  uint8  
 14  35-54 years           

---
---
# Multiple Regression

Tendo a preparação da base de dados completa, podemos iniciar o estudo da relação das variáveis e assim, prever o número de pessoas a cometer suicídio em um ano.

---
Para iniciar a aplicação, é necessário separar os dados independentes dos dados dependentes.

Para isto, "X" foi declarado como sendo as variáveis independentes, e "y" foi definido como o valor dependente. 

In [14]:
X = suicidios[['15-24 years', '25-34 years', '35-54 years', '55-74 years', '75+ years', 'classificação econômica', 'Boomers', 'G.I. Generation', 'Generation X', 'Generation Z', 'Millenials', 'Silent', 'sexo']]
y = suicidios['suicides_no']

#fazer X_train e y_train
# Separar o dataset para ser utilizado no teste e no treinamento
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [15]:
modelo = linear_model.LinearRegression().fit(x_train,y_train)
modelo.score(x_train,y_train)

0.05851148178499033

In [16]:
modelo.score(x_test,y_test)

0.062207255832034014

In [18]:
print(modelo.coef_)

[ 1.34257561e+02  1.80870208e+02  4.47186337e+02  2.91886779e+02
  8.07106472e+01 -1.49452847e+02 -2.73075583e+13 -2.73075583e+13
 -2.73075583e+13 -2.73075583e+13 -2.73075583e+13 -2.73075583e+13
  2.54884659e+02]


realizar split para separar train e test

In [19]:
previsto = modelo.predict([[0,1,0,0,0,2,0,0,0,1,0,0,1]])
previsto

array([419.43359375])

In [20]:
'''
insert graph depois
'''

'\ninsert graph depois\n'

# Bibliografia

_links utilizados:_

- https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
- https://www.imf.org/en/Publications/WEO/weo-database/2021/April/weo-report?c=512,914,612,614,311,213,911,314,193,122,912,313,419,513,316,913,124,339,638,514,218,963,616,223,516,918,748,618,624,522,622,156,626,628,228,924,233,632,636,634,238,662,960,423,935,128,611,321,243,248,469,253,642,643,939,734,644,819,172,132,646,648,915,134,652,174,328,258,656,654,336,263,268,532,944,176,534,536,429,433,178,436,136,343,158,439,916,664,826,542,967,443,917,544,941,446,666,668,672,946,137,546,674,676,548,556,678,181,867,682,684,273,868,921,948,943,686,688,518,728,836,558,138,196,278,692,694,962,142,449,564,565,283,853,288,293,566,964,182,359,453,968,922,714,862,135,716,456,722,942,718,724,576,936,961,813,726,199,733,184,524,361,362,364,732,366,144,146,463,528,923,738,578,537,742,866,369,744,186,925,869,746,926,466,112,111,298,927,846,299,582,487,474,754,698,&s=NGDPD,PPPGDP,NGDPDPC,PPPPC,&sy=2010&ey=2021&ssm=0&scsm=1&scc=0&ssd=1&ssc=0&sic=1&sort=subject&ds=.&br=1
- https://www.w3schools.com/python/python_ml_multiple_regression.asp
- https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
- 